## Import Modules

In [1]:
import litebird_sim as lbs
import numpy as np
from litebird_sim.hwp_sys.hwp_sys import compute_orientation_from_detquat
from litebird_sim import mpi
import matplotlib.pyplot as plt
import healpy as hp

## Define Parameters

In [ ]:
start_time = 0
time_span_s = 365 * 24 * 3600
nside = 128
sampling = 19
hwp_radpsec = lbs.IdealHWP(
    46 * 2 * np.pi / 60,
).ang_speed_radpsec

lbs.PTEP_IMO_LOCATION = "/.path/to/schema.json"
imo = lbs.Imo(flatfile_location=lbs.PTEP_IMO_LOCATION)

## Create Simulation

In [3]:
sim = lbs.Simulation(
    start_time=start_time, duration_s=time_span_s, random_seed=0, imo=imo
)

comm = sim.mpi_comm

## Define Frequency Channel

In [4]:
channelinfo = lbs.FreqChannelInfo(
    bandcenter_ghz=140.0,
    channel="L4-140",
    bandwidth_ghz=42.0,
    net_detector_ukrts=38.44,
    net_channel_ukrts=3.581435543962163,
    pol_sensitivity_channel_ukarcmin=7.24525963532118,
)

## Define Scanning Strategy

In [5]:
scan_strat = lbs.SpinningScanningStrategy(
    spin_sun_angle_rad=np.deg2rad(45.0),
    precession_rate_hz=1.0 / (60.0 * 192.348),
    spin_rate_hz=0.05 / 60.0,
)

sim.set_scanning_strategy(append_to_report=False, scanning_strategy=scan_strat)

## Define Instrument

In [6]:
instr = lbs.InstrumentInfo(
    name="LFT",
    boresight_rotangle_rad=0.0,
    spin_boresight_angle_rad=0.8726646259971648,
    spin_rotangle_rad=0.0,
    hwp_rpm=46.0,
    number_of_channels=1,
)

sim.set_instrument(instr)

## Set HWP

In [7]:
sim.set_hwp(lbs.IdealHWP(hwp_radpsec))

## Define Detectors

In [8]:
dets = []

quats = [
    [0.03967584136504414, 0.03725809501267564, 0.0, 0.9985177324254199],
    [0.05440050811606006, -0.001709604840948807, 0.706058659733029, 0.7060586597330291],
]


for d in range(1):
    det = lbs.DetectorInfo.from_dict(
        {
            "channel": channelinfo,
            "bandcenter_ghz": 140.0,
            "sampling_rate_hz": sampling,
            "quat": quats[d],
        }
    )
    det.phi = 0
    det.pol_angle_rad = compute_orientation_from_detquat(quats[d]) % (2 * np.pi)
    det.mueller_hwp = {
        "0f": np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]], dtype=np.float32),
        "2f": np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]], dtype=np.float32),
        "4f": np.array([[0, 0, 0], [1.65e-5, 1, 1], [1.65e-5, 1, 1]], dtype=np.float32),
    }
    # In this case we don't define mueller_hwp_solver, so the matrix for the mapmaking on-the-fly will be
    # the one for the ideal rotating hwp.

    dets.append(det)

## Create Observations and Prepare Pointings

In [9]:
(obs,) = sim.create_observations(
    detectors=dets,
)

sim.prepare_pointings(append_to_report=False)

## Generate Input Sky Maps

In [11]:
if comm.rank == 0:
    Mbsparams = lbs.MbsParameters(
        make_cmb=True,
        seed_cmb=1234,
        make_noise=False,
        make_dipole=True,
        make_fg=True,
        fg_models=["pysm_synch_0", "pysm_dust_0", "pysm_freefree_1"],
        gaussian_smooth=True,
        bandpass_int=False,
        maps_in_ecliptic=True,
        nside=nside,
        units="K_CMB",
    )

    mbs = lbs.Mbs(simulation=sim, parameters=Mbsparams, channel_list=[channelinfo])

    input_maps = mbs.run_all()[0]["L4-140"]

else:
    input_maps = None

if mpi.MPI_ENABLED:
    input_maps = comm.bcast(input_maps, root=0)

[2025-03-05 14:03:30,204 INFO MPI#0000] generating and saving cmb simulations
[2025-03-05 14:03:30,230 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-03-05 14:03:30,231 INFO MPI#0000] -> fwhm is 0.000000 arcmin
[2025-03-05 14:03:30,353 INFO MPI#0000] Access data from /home/mgomes/litebird/litebird_sim/litebird_sim/hwp_sys/cmb/0000/cmb_0000_date_250305.fits
[2025-03-05 14:03:30,355 INFO MPI#0000] NSIDE = 128
[2025-03-05 14:03:30,356 INFO MPI#0000] ORDERING = RING in fits file
[2025-03-05 14:03:30,356 INFO MPI#0000] INDXSCHM = IMPLICIT
[2025-03-05 14:03:30,542 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-03-05 14:03:30,543 INFO MPI#0000] -> fwhm is 0.000000 arcmin
[2025-03-05 14:03:30,544 INFO MPI#0000] Sigma is 0.000000 arcmin (0.000000 rad) 
[2025-03-05 14:03:30,544 INFO MPI#0000] -> fwhm is 0.000000 arcmin
[2025-03-05 14:03:30,552 INFO MPI#0000] generating and saving fg simulations
[2025-03-05 14:03:30,556 INFO MPI#0000] Retrieve data for pysm_2/synch_t

## Generate HwpSys object and Set its parameters

In [12]:
hwp_sys = lbs.HwpSys(sim)

hwp_sys.set_parameters(
    nside=nside,
    maps=input_maps,
    Channel=channelinfo,
    Mbsparams=Mbsparams,
    integrate_in_band=False,
    integrate_in_band_solver=False,
    build_map_on_the_fly=True,
    comm=comm,
)

## Fill TOD and perform Mapmaking on-the-fly

In [ ]:
hwp_sys.fill_tod(
    observations=[obs],
    input_map_in_galactic=False,
)

output_maps = hwp_sys.make_map([obs])

## Show Leakage Maps

In [ ]:
plt.figure(figsize=(32, 10))
stokes_parameters = ["T", "Q", "U"]
for i in range(3):
    hp.mollview(
        (output_maps[i] - input_maps[i]) * 1e6,
        cmap="jet",
        min=-0.025,
        max=0.025,
        sub=(1, 3, i + 1),
        title="Leakage " + stokes_parameters[i],
    )